In [ ]:
import sys
sys.path.append('./RECOTOOLS/')

import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import sparse
import scipy
import scipy as sp
import matplotlib.pyplot as plt

from core import system_run, survey_metadata, run_collection, feature_collection
import datalibrary
import processor
import systems.pelisystem
from core import system_run, survey_metadata, run_collection
from pandas_tools import *
from mugrass.units import *
from mugrass.voxelframe_meshgrid import *
from mugrass.units import *
from mugrass.voxelframe_meshgrid import *
from mugrass.pixel_collection import *
from mugrass.linear_solver_sparse import *
from mugrass.muon_opacity import *
from mugrass.units import *
from mugrass.muon_opacity import *
from mugrass.units import *

from numba import jit

In [ ]:
# Load in pixel dataframe calculated from survey data (made using 4_Convert_Features_to_Pixels.ipynb)
pixels  = pixel_collection(pickle_path="pixel_dataframe.pxl")

# Load in custom voxel dataframe (made using 5_Build_Test_Grid.ipynb)
voxels  = voxelframe_meshgrid(pickle_path="voxel_dataframe_masked.vxl")

# Load in weights matrix in CRS and CCS sparse formats, calculated by 6_Run_Embree_Calculator.py
weights_crs=scipy.sparse.load_npz(f"weights_masked_crs.npz")
weights_ccs=scipy.sparse.load_npz(f"weights_masked_ccs.npz")


# Get the set of sparse vectors
V_ccs=getattr(weights_ccs,"data")
I_ccs=getattr(weights_ccs,"indices")
S_ccs=getattr(weights_ccs,"indptr")

V_crs=getattr(weights_crs,"data")
I_crs=getattr(weights_crs,"indices")
S_crs=getattr(weights_crs,"indptr")

handler = linear_solver_sparse(voxels=voxels,pixels=pixels,v_crs=V_crs,i_crs=I_crs,s_crs=S_crs,v_ccs=V_ccs,i_ccs=I_ccs,s_ccs=S_ccs)





In [ ]:

# View sample pixel FOV slices


def crs_getrow(i,V_crs,I_crs,S_crs,S_ccs):
    n_cols=len(S_ccs)-1
    row=np.zeros(n_cols)
    for v in range(S_crs[i],S_crs[i+1]):
        row[I_crs[v]]=V_crs[v]
    return row


for i in range(len(pixels.df)):
    #print(f"pixel {i} out of {len(pixels.df)}")
    pixel_weight = (crs_getrow(i,V_crs,I_crs,S_crs,S_ccs)).transpose()

    voxels.df["weight"] = pixel_weight
    print(f"pixel {i} out of {len(pixels.df)}, x={pixels.df.x[i]}, combo={pixels.df.combo[i]}:")
    plt.hist2d(x=voxels.df.x, y=voxels.df.z, weights=voxels.df["weight"]>0, bins=[len(np.unique(voxels.df.x)),len(np.unique(voxels.df.z))])
    
    plt.show()
    
    if i > 3: break
    #break



In [ ]:
# Prepare SART prerequisites

from numba import jit
@jit
def Prepare_loopfunc(V_ccs,I_ccs,S_ccs,S_crs):

    n_rows=len(S_crs)-1
    n_voxels=len(S_ccs)-1

    sart_f = np.zeros(n_voxels)

    for i in range(n_voxels): 
        col=np.zeros(n_rows)
        for s in range(S_ccs[i],S_ccs[i+1]):
            col[I_ccs[s]]=V_ccs[s]
        sart_f[i]=col.sum()
        if sart_f[i] != 0:
            sart_f[i]=1.0/sart_f[i]
            #print(sart_f[i])    
    return sart_f


def SART_Prepare(handler):

    n_rows=len(handler.data["pixels"])

    n_voxels=len(handler.data["voxels"])

    #weights_sq_vec=np.zeros(n_rows)

    V_crs=handler.data["v_crs"]
    I_crs=handler.data["i_crs"]
    S_crs=handler.data["s_crs"]

    V_ccs=handler.data["v_ccs"]
    I_ccs=handler.data["i_ccs"]  
    S_ccs=handler.data["s_ccs"]

    sart_p = np.zeros(n_rows)
    
    for i in range(n_rows): 
        row=crs_getrow(i,V_crs,I_crs,S_crs,S_ccs)
        #weights_sq_vec[i]=row.dot(row.transpose())
        sart_p[i]=row.sum()
        if sart_p[i] != 0:
            sart_p[i]=1.0/sart_p[i]

    
    sart_f=Prepare_loopfunc(V_ccs,I_ccs,S_ccs,S_crs)
    
    handler.data["SART_P"]=sart_p
    handler.data["SART_F"]=sart_f

    #handler.data["weights_squared"]=weights_sq_vec

    print("SART_P",handler.data["SART_P"].shape, handler.data["SART_P"].ndim)
    print("SART_F",handler.data["SART_F"].shape, handler.data["SART_F"].ndim)
    


handler.Prepare()

SART_Prepare(handler)


In [ ]:

@jit
def Predict_loopfunc(I_crs,S_crs,V_crs,voxelsdt):
    n_pixels=len(S_crs)-1
    prediction=np.zeros(n_pixels)
    for m in range(n_pixels):
        #print(f"row {m} out of {n_pixels}")
        for v in range(S_crs[m],S_crs[m+1]):
            prediction[m] += V_crs[v]*voxelsdt[I_crs[v]]
    prediction[np.isnan(prediction)] = 0.0
    return prediction

@jit
def Update_loopfunc(V_ccs,I_ccs,S_ccs,pixels,prediction,Pnp,Fnp):

    n_voxels=len(S_ccs)-1

    update=np.zeros(n_voxels)

    for v in range(n_voxels):

        delta_rho_v=0.

        for s in range(S_ccs[v],S_ccs[v+1]):
            m_nz=I_ccs[s]
            #delta_rho_v += 1.0*(pixels[m_nz]-prediction[m_nz])*V_ccs[s]*Pnp[m_nz]
            delta_rho_v += (pixels[m_nz]-prediction[m_nz])*V_ccs[s]*Pnp[m_nz]

        delta_rho_v *= Fnp[v]
        update[v]=delta_rho_v


    return update * 0.9999

def SART_Predict(handler):
    #print("prediction start")
    voxelsdt = handler.data["voxels"]["value"].copy()
    V_crs=handler.data["v_crs"]
    I_crs=handler.data["i_crs"]
    S_crs=handler.data["s_crs"]

    V_ccs=handler.data["v_ccs"]
    I_ccs=handler.data["i_ccs"]  
    S_ccs=handler.data["s_ccs"]
    #n_pixels=len(S_crs)-1

    #prediction=np.zeros(n_pixels)   
    voxelsdt = handler.data["voxels"]["value"].copy()
    voxels=voxelsdt.to_numpy()

    prediction=Predict_loopfunc(I_crs,S_crs,V_crs,voxels)

    #print("prediction end")
    return prediction

def SART_Update(handler):
    prediction = SART_Predict(handler)

    pixelsdt = handler.data["pixels"]["opacity"].copy()    
    
    pixels=pixelsdt.to_numpy()

    V_crs=handler.data["v_crs"]
    I_crs=handler.data["i_crs"]
    S_crs=handler.data["s_crs"]

    V_ccs=handler.data["v_ccs"]
    I_ccs=handler.data["i_ccs"]  
    S_ccs=handler.data["s_ccs"]

    Pnp = np.copy(handler.data['SART_P'])
    Fnp = np.copy(handler.data['SART_F'])
    
    update=Update_loopfunc(V_ccs,I_ccs,S_ccs,pixels,prediction,Pnp,Fnp)

    return update

handler.Reset()


voxels.df.loc[ voxels.df.zu <= voxels.df["DTM_height"], 'value' ] = 2.65*g/cm3

voxels.df.loc[ voxels.df.zu > voxels.df["DTM_height"], 'value' ] = 0.0


tunnel_height = 6*m # radius
tunnel_width = 9*m/2 # radius

voxels.df["in_tunnel"] = ((voxels.df.y/tunnel_width)**2 + (voxels.df.z/tunnel_height)**2) < 1.0
voxels.df.loc[voxels.df.in_tunnel, "value"] = 0.0


voxels.df["initial_guess"] = voxels.df["value"]

slice = voxels.df[ (np.abs(voxels.df.y) < 1*m)]

plt.hist2d(x=slice.x/m, y=slice.z/m, weights=(slice.initial_guess/float(len(np.unique(slice.y))))/(g/cm3), bins=[len(np.unique(slice.x)),len(np.unique(slice.z))])
plt.title("Initial guess")
plt.xlabel("Position [m]")
plt.ylabel("Elevation [m]")
plt.colorbar(label="Density (g/cm3)")
plt.show()

plt.scatter(x=pixels.df["x"]/m,y=pixels.df["opacity"]/(g/cm2))
plt.xlabel("Position [m]")
plt.ylabel("Estimated Opacity [g/cm2]")
plt.show()


# Call a SART Update and see how our solution looks
n_iterations = 5


# incorporate prior knowledge if desired
use_priorknowledge=0

pk_cutoff=1.25*g/cm3


# Run SART
for i in range(n_iterations):

    print(f"SART iter {i}")
    voxels.df.value += SART_Update(handler)
    voxels.df.value[np.isnan(voxels.df.value)] = 2.65*g/cm3
    voxels.df.loc[(voxels.df.value < 0), 'value']=0.0*g/cm3
    voxels.df.loc[(voxels.df.value > 2.65*g/cm3), 'value']=2.65*g/cm3

    # Force voxels below cut to zero
    if (i % 1 == 0) & use_priorknowledge:
        voxels.df.loc[(voxels.df.value < pk_cutoff) & (voxels.df.zu <= voxels.df["DTM_height"]), 'value']=0.0*g/cm3

   
    

# Draw 1m slice along crown
slice = voxels.df[ (np.abs(voxels.df.y) < 1*m)] 
plt.hist2d(x=slice.x/1000, y=slice.z/1000, weights=(slice.value/float(len(np.unique(slice.y))))/(g/cm3), bins=[len(np.unique(slice.x)),len(np.unique(slice.z))])#,cmin=0,cmax=0.005)

if use_priorknowledge:
    plt.title(f"Solution, {n_iterations} iterations, pk cut = {pk_cutoff/(g/cm3):.2f} g/cm3")
else:
    plt.title(f"Solution, {n_iterations} iterations")

plt.xlabel("Position [m]")
plt.ylabel("Elevation [m]")
plt.colorbar(label="Density (g/cm3)")
plt.show()


voxels.df.to_csv("sartoutput.csv")   